In [1]:
#Libraries
import pandas as pd
import numpy as np
from etl.utils import read_sql_table
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

In [27]:
df = read_sql_table("gold_cpw")

###Variable manipulation
# Convert HourDK to datetime if it's not already in datetime format
df['hour_utc'] = pd.to_datetime(df['hour_utc'])

# Extract the hour from the HourDK column
df['hour'] = df['hour_utc'].dt.hour

#Subsetting variables
df = df[['hour', 'consumption_kwh', 'spot_price_dkk', 'temp_mean_past1h', 'wind_speed_past1h',
    'humidity_past1h', 'precip_past1h']]
#Converting variables to float64
df['hour'] = df['hour'].astype('float64')
df['spot_price_dkk'] = df['spot_price_dkk'].astype('float64')

print(df.head().to_string(), "\n")
print(df.info())

   hour  consumption_kwh  spot_price_dkk  temp_mean_past1h  wind_speed_past1h  humidity_past1h  precip_past1h
0   0.0       143408.914      383.950012               5.2                3.7             88.0            0.0
1   1.0       136154.493      387.230011               4.5                3.9             91.0            0.0
2   2.0       130648.111      386.929993               3.8                4.0             93.0            0.0
3   3.0       126554.756      387.829987               3.2                4.2             95.0            0.0
4   4.0       131530.892      391.480011               3.3                4.1             96.0            0.0 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 647 entries, 0 to 646
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   hour               647 non-null    float64
 1   consumption_kwh    647 non-null    float64
 2   spot_price_dkk     647 non-null    fl

In [28]:
##Checking for missing values
print(df.isna().any())
#No missing values

hour                 False
consumption_kwh      False
spot_price_dkk       False
temp_mean_past1h     False
wind_speed_past1h    False
humidity_past1h      False
precip_past1h        False
dtype: bool


In [29]:
##Splitting dataset
X = df[['hour', 'spot_price_dkk', 'temp_mean_past1h', 'wind_speed_past1h', 'humidity_past1h', 'precip_past1h']]
y = df[['consumption_kwh']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
print(X_train)

     hour  spot_price_dkk  temp_mean_past1h  wind_speed_past1h  \
171   3.0      494.239990              -1.0                1.1   
477  21.0      716.400024               4.6                1.4   
249   9.0      461.429993               2.4                6.2   
574  22.0      469.649994               4.1                2.0   
185  17.0      624.400024               0.1                1.7   
..    ...             ...               ...                ...   
98    2.0      420.089996               2.4                4.5   
322  10.0      371.649994               8.7                5.2   
382  22.0      422.209991               1.5                3.3   
365   5.0      352.410004               5.7                4.4   
510   6.0      477.450012               6.9                4.2   

     humidity_past1h  precip_past1h  
171             97.0            0.0  
477             99.0            0.0  
249             83.0            0.0  
574             97.0            0.1  
185             9

In [30]:
####Linear Regression####
##Training and evaluating linear regression model without preprocessing
est = sm.OLS(y_train, sm.add_constant(X_train)) #model with constant
est_fit = est.fit()
print(est_fit.summary()) #All but hour and precip_past1h are significant
#In a zero-intercept model, all but precip_past1h and spot_price_dkk are significant


                            OLS Regression Results                            
Dep. Variable:        consumption_kwh   R-squared:                       0.139
Model:                            OLS   Adj. R-squared:                  0.129
Method:                 Least Squares   F-statistic:                     13.76
Date:                Fri, 03 May 2024   Prob (F-statistic):           1.64e-14
Time:                        11:02:28   Log-Likelihood:                -6404.1
No. Observations:                 517   AIC:                         1.282e+04
Df Residuals:                     510   BIC:                         1.285e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               2.65e+05   2.98e+0

In [36]:
#Linear regression with scaling
X_train_scale = StandardScaler().fit_transform(X_train)
est = sm.OLS(y_train, sm.add_constant(X_train_scale))
est_fit = est.fit()
print(est_fit.summary())
#Conclude on variable importance based on coefficients...

                            OLS Regression Results                            
Dep. Variable:        consumption_kwh   R-squared:                       0.139
Model:                            OLS   Adj. R-squared:                  0.129
Method:                 Least Squares   F-statistic:                     13.76
Date:                Fri, 03 May 2024   Prob (F-statistic):           1.64e-14
Time:                        11:27:59   Log-Likelihood:                -6404.1
No. Observations:                 517   AIC:                         1.282e+04
Df Residuals:                     510   BIC:                         1.285e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.006e+05   2567.816     78.117      0.0

In [37]:
###Prediction accuracy - linear regression
X_test_scale = StandardScaler().fit_transform(X_test)
y_pred = est_fit.predict(sm.add_constant(X_test_scale))
mse = mean_squared_error(y_test, y_pred)
print(round(mse)) #3498841479

3498841479


In [ ]:
###Random Forest
rf_model = RandomForestClassifier()
